Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-9.0057e-02,  3.1231e-01, -2.7692e-01, -3.2546e-01, -3.7012e-01,
         -2.4185e-01],
        [-2.1187e-01,  1.2767e-01, -4.8976e-02,  3.4051e-01, -4.6319e-02,
         -2.4523e-01],
        [ 3.9700e-01, -3.9724e-02, -1.7884e-01, -1.0022e-01, -2.1898e-01,
         -1.1990e-01],
        [-2.0903e-01,  4.8217e-02, -3.9444e-01, -2.8280e-02,  1.0678e-01,
         -3.5748e-01],
        [ 2.1948e-03, -3.0103e-01,  2.6996e-01, -9.0454e-02, -7.5002e-02,
         -1.0879e-01],
        [ 1.6651e-01,  3.4832e-01,  1.1682e-01, -2.0790e-01, -5.1887e-02,
         -3.8847e-01],
        [ 1.7228e-01,  1.4091e-01, -3.8557e-01,  1.1776e-01, -1.0376e-01,
         -1.1607e-01],
        [-2.4074e-01, -3.5588e-01, -1.6152e-01,  3.8291e-01, -3.1749e-01,
          3.1401e-01],
        [-8.6887e-02, -2.8878e-01,  5.5152e-02,  1.8590e-01, -2.9172e-01,
          3.7107e-01],
        [-1.1605e-01, -3.3013e-01,  2.8206e-03, -9.8544e-02,  8.1680e-02,
          2.0496e-01],
        

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.1771],
        [-0.1743],
        [-0.1749],
        ...,
        [-0.1743],
        [-0.1775],
        [-0.1764]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1833)
1 tensor(0.1776)
2 tensor(0.1721)
3 tensor(0.1666)
4 tensor(0.1611)
5 tensor(0.1556)
6 tensor(0.1502)
7 tensor(0.1447)
8 tensor(0.1392)
9 tensor(0.1338)
10 tensor(0.1283)
11 tensor(0.1229)
12 tensor(0.1175)
13 tensor(0.1123)
14 tensor(0.1075)
15 tensor(0.1027)
16 tensor(0.0979)
17 tensor(0.0930)
18 tensor(0.0881)
19 tensor(0.0831)
20 tensor(0.0780)
21 tensor(0.0730)
22 tensor(0.0682)
23 tensor(0.0637)
24 tensor(0.0594)
25 tensor(0.0554)
26 tensor(0.0518)
27 tensor(0.0486)
28 tensor(0.0457)
29 tensor(0.0432)
30 tensor(0.0412)
31 tensor(0.0395)
32 tensor(0.0382)
33 tensor(0.0373)
34 tensor(0.0367)
35 tensor(0.0365)
36 tensor(0.0364)
37 tensor(0.0366)
38 tensor(0.0368)
39 tensor(0.0371)
40 tensor(0.0374)
41 tensor(0.0377)
42 tensor(0.0379)
43 tensor(0.0379)
44 tensor(0.0378)
45 tensor(0.0377)
46 tensor(0.0374)
47 tensor(0.0370)
48 tensor(0.0366)
49 tensor(0.0361)
50 tensor(0.0356)
51 tensor(0.0352)
52 tensor(0.0347)
53 tensor(0.0343)
54 tensor(0.0339)
55 tensor(0.0335)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1714.50405502]
[1409.99999118] [1473.52832222]
[1128.99997044] [1078.60461426]
[12991.99967575] [12018.29267883]
[14720.0001564] [14468.25957489]
[2938.99994564] [2815.03854275]
[6635.00014687] [8023.87178802]
[11215.99973297] [9561.57770157]
[1040.99999189] [1498.50454617]
[5040.00003624] [5440.48228455]
[9541.99988747] [8300.82865143]
[3792.99988461] [5470.80951309]
[1002.0000155] [762.25096226]
[2542.00005341] [1593.10253525]
[6788.00001717] [6624.26947403]
[9386.00025749] [7934.11492157]
[1245.99996853] [1644.72932529]
[997.00000262] [558.7702961]
[708.00000262] [789.21903324]
[1435.99997544] [1527.44103718]
[2590.00006676] [2552.69076252]
[1392.99997497] [1787.37969398]
[11755.99946976] [9671.7981987]
[2007.99997377] [1374.60625839]
[2707.99998474] [2612.69642925]
[631.00000417] [774.87646675]
[652.99999881] [1048.99094486]
[2492.00006247] [2689.20022297]
[594.99999416] [579.57816601]
[760.0000056] [1100.00063992]
[1179.00003028] [1031.0315237]
[1789.9999361] [196

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [853.00940228]
[628.00000334] [634.84385967]
[442.00000328] [744.04156113]
[2284.99997044] [2618.56310368]
[1040.99999189] [846.61851406]
[809.99999654] [1136.70607853]
[3049.99987316] [3455.77386951]
[720.00000596] [753.34539127]
[877.99999249] [598.95312977]
[1292.00000429] [1734.52725792]
[7056.00004578] [4966.88406372]
[4176.99999142] [5801.60219574]
[13701.00020599] [14958.80241394]
[2948.0000515] [4045.61139965]
[16732.99985504] [14477.80346298]
[1815.00000048] [1950.92585278]
[16220.00016022] [12325.29895401]
[5967.00019073] [4817.99770069]
[473.00000042] [585.50327015]
[1078.9999795] [965.30332756]
[452.00000319] [690.58084965]
[801.99998283] [808.473279]
[2458.99994993] [2558.07787228]
[2492.99998236] [2642.14280224]
[582.00000203] [710.52467632]
[1405.9999671] [1579.98974895]
[8715.99991035] [11762.6858139]
[7634.00004578] [6608.50006485]
[506.99999839] [698.54713535]
[665.99999094] [761.50184917]
[1815.00000048] [1601.31769848]
[707.00001383] [630.10223293]
[5

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1373.9903924356233


## Conclusão



Com esse experimento podemos perceber que o `pytorch` é uma biblioteca muito útil para a criação de uma rede neural, podendo com ela criar muito mais reapidamente e treina-la mais rapidamente tambem, mas tmabem é necessario lembrar que ela não cria a rede sozinha, que está escrevendo o codigo precisa saber o basico de uma rede, para assim criar as funções basicas e definir os seus parametros. Agora a partir deste experimento podemos dizer que compreendemos como é criar uma rede neural do zero, utilizando classe, e com o auxilio de uma biblioteca, neste caso o `pytorch`.

## Playground

